In [1]:
import pandas as pd
from pathlib import Path
import xarray as xr

from opera_tropo import checks

import logging
logger = logging.getLogger(__name__)

In [2]:
import sys
# Method 1: Basic setup (simplest)
def setup_basic_logging():
    """Basic logging setup for Jupyter notebook"""
    logging.basicConfig(
        level=logging.INFO,
        format='%(levelname)s - %(name)s - %(message)s',
        handlers=[logging.StreamHandler(sys.stdout)]
    )

In [3]:
setup_basic_logging()

In [4]:
work_dir = Path('/u/aurora-r0/govorcin/01_OPERA/TROPO/final/tropo/notebooks/download_test').absolute()
filename = 'ECMWF_TROP_202409100600_202409100600_1.nc'

In [5]:
# Print the valid range
for id, key in checks.VALID_RANGE.items():
    print(f'{id} - {key}')

t - [140, 360]
q - [0, 0.3]
z - [-5000, 70000]
lnsp - [10.2, 11.75]


In [6]:
ds = xr.open_dataset(work_dir / filename, chunks={'level':-1})
#ds = ds.chunk({, 'longitude':256, 'latitude':256})

In [7]:
from dask.distributed import Client

client = Client(
    #n_workers=4,
    #threads_per_worker=2,
    #memory_limit=f"{30}GB",
    )
client.dashboard_link

INFO - distributed.http.proxy - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO - distributed.scheduler - State start
INFO - distributed.diskutils - Found stale lock file and directory '/tmp/dask-scratch-space-75956/scheduler-bj59spjl', purging
INFO - distributed.scheduler -   Scheduler at:     tcp://127.0.0.1:45089
INFO - distributed.scheduler -   dashboard at:  http://127.0.0.1:8787/status
INFO - distributed.scheduler - Registering Worker plugin shuffle
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:45780'
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:33922'
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:37921'
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:44793'
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:34331'
INFO - distributed.nanny -         Start Nanny at: 'tcp://127.0.0.1:33407'
INFO - dis

'http://127.0.0.1:8787/status'

INFO - distributed.nanny - Closing Nanny gracefully at 'tcp://127.0.0.1:40534'. Reason: worker-close
INFO - distributed.nanny - Closing Nanny gracefully at 'tcp://127.0.0.1:46842'. Reason: worker-close
INFO - distributed.core - Received 'close-stream' from tcp://127.0.0.1:38990; closing.
INFO - distributed.nanny - Closing Nanny gracefully at 'tcp://127.0.0.1:37351'. Reason: worker-close
INFO - distributed.nanny - Closing Nanny gracefully at 'tcp://127.0.0.1:37921'. Reason: worker-close
INFO - distributed.nanny - Closing Nanny gracefully at 'tcp://127.0.0.1:44793'. Reason: worker-close
INFO - distributed.core - Received 'close-stream' from tcp://127.0.0.1:38968; closing.
INFO - distributed.nanny - Closing Nanny gracefully at 'tcp://127.0.0.1:35071'. Reason: worker-close
INFO - distributed.nanny - Closing Nanny gracefully at 'tcp://127.0.0.1:38791'. Reason: worker-close
INFO - distributed.nanny - Closing Nanny gracefully at 'tcp://127.0.0.1:34331'. Reason: worker-close
INFO - distributed

In [8]:
%%time
# Preform checks
checks.validate_input(ds)

INFO - opera_tropo.checks - Performing checkup of input file
INFO - opera_tropo.checks -   Checking coordinate ranges and data variables.
INFO - opera_tropo.checks -   Checking nans and data valid range.
INFO - opera_tropo.checks -    Variable "q" stats: min = 0.00000, max = 0.02865, NaNs = 0
INFO - opera_tropo.checks -    Variable "t" stats: min = 182.19893, max = 317.13504, NaNs = 0
INFO - opera_tropo.checks -    Variable "lnsp" stats: min = 10.74795, max = 11.57485, NaNs = 0
INFO - opera_tropo.checks -    Variable "z" stats: min = -4563.28223, max = 62700.71875, NaNs = 0
CPU times: user 2.07 s, sys: 1.48 s, total: 3.55 s
Wall time: 19.2 s


<xarray.Dataset> Size: 29GB
Dimensions:    (time: 1, level: 137, latitude: 2560, longitude: 5120)
Coordinates:
  * longitude  (longitude) float32 20kB 2.501e-07 0.07031 0.1406 ... 359.9 359.9
  * latitude   (latitude) float32 10kB 89.95 89.88 89.81 ... -89.88 -89.95
  * level      (level) int32 548B 1 2 3 4 5 6 7 ... 131 132 133 134 135 136 137
  * time       (time) datetime64[ns] 8B 2024-09-10T06:00:00
Data variables:
    z          (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
    t          (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
    q          (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
    lnsp       (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2025-05-21 02:32:18 GMT by grib_to_netcdf-2.31.0: grib_to_n...

In [9]:
%%time
ds = xr.open_dataset('/u/aurora-r0/govorcin/01_OPERA/TROPO/data/ECMWF_TROP_201606230600_201606230600_1.nc', chunks={})
vars_out = checks.check_nans_valid_range(ds)

WARNING - opera_tropo.checks -    Variable "q" is out of valid range [0, 0.3]: min = -0.00001 [<0], max = 1.69043 [>0.3]
WARNING - opera_tropo.checks -    Variable "t" is out of valid range [140, 360]: min = 82.57539 [<140], max = 606.28271 [>360]
INFO - opera_tropo.checks -    Variable "lnsp" stats: min = 10.74720, max = 11.57020, NaNs = 0
INFO - opera_tropo.checks -    Variable "z" stats: min = -4418.77490, max = 62685.22656, NaNs = 0
CPU times: user 3.04 s, sys: 825 ms, total: 3.87 s
Wall time: 8.12 s


In [22]:
client.close()

In [9]:
%%time
# Test it out
ds['q'] = ds.q.where(ds.q >= 0.15, -0.1)
checks.validate_input(ds)

INFO - opera_tropo.checks - Performing checkup of input file
INFO - opera_tropo.checks -   Checking coordinate ranges and data variables.
INFO - opera_tropo.checks -   Checking nans and data valid range.


INFO - opera_tropo.checks - Variable "lnsp" stats: min = 10.747946739196777, max = 11.574851036071777, NaNs = 0
INFO - opera_tropo.checks - Variable "t" stats: min = 182.1989288330078, max = 317.1350402832031, NaNs = 0
WARNING - opera_tropo.checks - Variable "q" is out of valid range [0, 0.5]: min = -0.10000 [<0], max = -0.10000 [>0.5]
INFO - opera_tropo.checks - Variable "z" stats: min = -4563.2822265625, max = 62700.71875, NaNs = 0
INFO - opera_tropo.checks - Specific humidify
INFO - opera_tropo.checks -   Clipping negative specific humidity.
CPU times: user 4.69 s, sys: 1.33 s, total: 6.01 s
Wall time: 37 s


<xarray.Dataset> Size: 29GB
Dimensions:    (time: 1, level: 137, latitude: 2560, longitude: 5120)
Coordinates:
  * longitude  (longitude) float32 20kB 2.501e-07 0.07031 0.1406 ... 359.9 359.9
  * latitude   (latitude) float32 10kB 89.95 89.88 89.81 ... -89.88 -89.95
  * level      (level) int32 548B 1 2 3 4 5 6 7 ... 131 132 133 134 135 136 137
  * time       (time) datetime64[ns] 8B 2024-09-10T06:00:00
Data variables:
    z          (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
    t          (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
    q          (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
    lnsp       (time, level, latitude, longitude) float32 7GB dask.array<chunksize=(1, 137, 512, 1024), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2025-05-21 02:32:18 GMT by grib_to_netcdf-2.31.0: grib_to_n...